In [2]:
# DONE: Hello World
# DONE: Crawl over stored Files
# DONE: Operate Elastic Search Class
# DONE: Populate wiki_evidence
# DONE: How to manage a peristant volume
# TODOs: Information Retrieval program, given search and rank sentences.
# TODOs: Implement DELETE on Database Class
# https://stackoverflow.com/questions/34621093/persist-elastic-search-data-in-docker-container
# TODOs: Repurpose DrQA Elasticsearch Module
# TODOs: Run Full-Evidence
# TODOs: Nice Logging

In [3]:
### CRAWL FILE-SYSTEM ###
import os
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

knowledge_path = "/Users/joshua.sheppard/full_wiki_extract"

def iter_filesys(path):
    if os.path.isfile(path):
        yield path

    elif os.path.isdir(path):
        for dir, _, filenames in os.walk(path):
                for f in filenames:
                    if not f.endswith('.DS_Store'):
                        yield os.path.join(dir, f)

    else:
        raise RuntimeError("Invalid path %s" % path)

kw_files = iter_filesys(knowledge_path)

kw_sample = []
for i in kw_files:
    kw_sample.append(i)

print(kw_sample)
print(len(kw_sample))

['/Users/joshua.sheppard/full_wiki_extract/GA/wiki_73', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_87', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_80', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_74', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_89', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_42', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_45', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_11', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_16', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_29', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_20', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_27', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_18', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_44', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_88', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_43', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_75', '/Users/joshua.sheppard/full_wiki_extract/GA/wiki_81', '/Users/j

In [4]:
### UPLOAD USING GENERATOR ###
from multiprocessing import Pool
from utils import get_contents
import utils
from tqdm import tqdm
import json

# TODOs: Apply Paralell Multiprocess
# def get_contents_(filename):
#     """Parse the contents of a file. Each line is a JSON encoded document."""
#     documents = []

#     with open(filename) as f:
#         for line in f:
#             doc = json.loads(line)

#             if doc["text"] == "": continue
#             if not doc: continue

#             documents.append((doc['id'], doc["title"], doc["text"]))

#     return documents

# files = [f for f in kw_sample]

# def generate_data():
#     with tqdm(total=len(files)) as pbar:
#         #for document in p.map(utils.get_contents_2, files):
#         for file in files:
#                 #yield get_contents_2(file)
#                 docs = get_contents_(file)
#                 for doc in docs:
#                     yield(doc)

#         pbar.update()

# generate_data()
# wiki_data = [i for i in generate_data()]

In [5]:
# len(wiki_data) 

In [5]:
# knowledge = wiki_data[0]
# knowledge

In [8]:
import json
from multiprocessing.pool import ThreadPool as Pool
from tqdm import tqdm
import spacy

files = [f for f in kw_sample]
nlp = spacy.load("en_core_web_sm")

def get_contents_(filename):
    """Parse the contents of a file. Each line is a JSON encoded document."""
    documents = []

    with open(filename) as f:
        for line in f:
            doc = json.loads(line)

            if doc["text"] == "": continue
            if not doc: continue

            documents.append((doc['id'], doc["title"], doc["text"]))

    return documents

def generate_data():
    #with tqdm(total=len(files)) as pbar:
        #for document in p.map(utils.get_contents_2, files):
        for file in files:
                #yield get_contents_2(file)
                docs = get_contents_(file)
                for doc in docs:
                    yield(doc)

        #pbar.update()

def sentence_window(article, window=3, step=2): 
    """ Generates a list of sentences of sliding size = window """
    
    sents = list(nlp(article).sents)
    
    if len(sents) == window:
        yield str(sents)

    for i in range(0, len(sents) - window + step):
        #print(sents[i:i + window])
        yield(sents[i:i + window])

# generate_data()
# wiki_data_passages = []
# count = 0

pool = Pool(8)

def passages(idx):
    count = 0
    with tqdm(total=len(files)) as pbar:
        for i in pool.apply(generate_data):
            count += 1
            id, title, article = i

            for paragraph in sentence_window(article):
                yield {
                    "_index": idx,
                    "document": {
                        "id": id,
                        "title": title,
                        "text": "".join([str(i).strip("\n").replace("\n", "") for i in paragraph])
                        }
                    }
            
            pbar.update()
     

In [9]:
# 36.2 = 1000
test = []

count = 0
for i in passages("testing"):
    count += 1
    test.append(i)
    
    if count > 100:
        break

  0%|          | 7/17000 [00:00<22:45, 12.44it/s]


In [17]:
len(test)
test

[{'_index': 'testing',
  'document': {'id': '61952548',
   'title': 'Dominic Ondoro',
   'text': "Dominic Ondoro (born March 3, 1988, also known as Pius Dominic Ondoro) is a Kenyan long-distance runner best known for winning the 2017 Houston Marathon in Texas.He has won many other major marathons, some multiple times, including four Twin Cities Marathon wins in Minneapolis-St. Paul, Minnesota, and two Melbourne Marathon wins in Australia.He is the course record holder at Twin Cities as well as Grandma's Marathon, where he broke hall of fame-runner Dick Beardsley's record in 2014."}},
 {'_index': 'testing',
  'document': {'id': '61952548',
   'title': 'Dominic Ondoro',
   'text': "He has won many other major marathons, some multiple times, including four Twin Cities Marathon wins in Minneapolis-St. Paul, Minnesota, and two Melbourne Marathon wins in Australia.He is the course record holder at Twin Cities as well as Grandma's Marathon, where he broke hall of fame-runner Dick Beardsley's 

In [18]:
from elasticsearch import Elasticsearch

# INIT OBJECT

PORT = "http://localhost:9200"
INDEX_NAME = "wiki_evidence"
errors_before_interrupt = 5
refresh_index_after_insert = False
max_insert_retries = 3
yield_ok = False

wiki_ev = Elasticsearch(
    PORT,
    #http_auth=(es_api_user, es_api_password)
    retry_on_timeout=True,  # should timeout trigger a retry on different node?
)

wiki_ev.elastic_index = INDEX_NAME

wiki_ev

<Elasticsearch(['http://localhost:9200'])>

In [39]:
#es = Elasticsearch("http://localhost:9200", retry_on_timeout=True)

In [19]:
import re
from elasticsearch import helpers
from elasticsearch.helpers import streaming_bulk, parallel_bulk
from tqdm import tqdm
import spacy

nlp = spacy.load("en_core_web_sm")

def sentence_window(article, window=3, step=2): 
    """ Generates a list of sentences of sliding size = window """
    
    sents = list(nlp(article).sents)
    
    if len(sents) == window:
        yield str(sents)

    for i in range(0, len(sents)):
        yield(str(sents[i:i + window]))

def load_data(docs, idx_):
    """ Generates an evidence document to be inserted into ES Index """
    for doc in docs:
        idx, title, text = doc

        for paragraph in sentence_window(text):
            yield {
                "_index": idx_,
                "document": {
                    "id": idx,
                    "title": title,
                    "text": paragraph
                }
            }

### TEST ###
#test = []
#id, title, text = knowledge

# kw = sentence_window(knowledge[2])
# for i in kw:
#     print(i)

es = wiki_ev
errors_count = 0

chunk_size = 25000
counta = len(files)//chunk_size

with tqdm(total=counta) as pbar:
    for ok, result in parallel_bulk(es, passages("wiki_evidence"), chunk_size=chunk_size, request_timeout=60*3):

        if ok is not True:
                logging.error('Failed to import data')
                logging.error(str(result))
                errors_count += 1

                if errors_count == errors_before_interrupt:
                    logging.fatal('Too many import errors, exiting with error code')
                    exit(1)
        
        pbar.update()   
        

0it [00:00, ?it/s]INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:2.422s]
1it [01:36, 96.35s/it]INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:2.503s]
50000it [03:22, 154.62it/s]INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:2.478s]
75000it [05:02, 202.46it/s]INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:2.670s]
100000it [06:32, 226.86it/s]INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:2.646s]
125000it [08:12, 240.94it/s]INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:2.446s]
150000it [09:42, 250.55it/s]INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:3.573s]
175000it [11:22, 253.10it/s]INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:3.139s]
200000it [12:52, 257.47it/s]INFO:elastic_t

In [ ]:
query = "government emails privacy"

def search(query, k=5):
    results = es.search(
        index = es.elastic_index,
        body= {
            "size": k,
            "query": {
                "match": {
                    "document.text": query,
        }}})

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    print(results)
    return (hits, doc_ids)


test = search(query, k=2)[0][0]["_source"]["document"]["text"]
test


In [ ]:
# import time
# from elasticsearch import helpers
# from elasticsearch.helpers import streaming_bulk, parallel_bulk
# from tqdm import tqdm

# "https://stackoverflow.com/questions/67522617/elasticsearch-bulk-insert-w-python-socket-timeout-error#:~:text=The%20connection%20to%20elasticsearch%20has,be%20handled%20as%20an%20error."

# "https://github.com/elastic/elasticsearch-py/issues/297"

# def load_data(docs, idx_):
#     for doc in docs:
#         idx, title, text = doc
#         doc_ = {"id": idx, "title": title, "text": text}

#         yield {
#             "_index": idx_,
#             "document": {
#                 "id": idx,
#                 "title": title,
#                 "text": text
#             }
#         }

# # TODOs: Utils, Duration Function as Decorator
# es = wiki_ev
# errors_count = 0

# # TODOs: Increase Chunk Size, with extended Timeout + handeling
# # TODOs: Experiment-Check with Yield OK
# for ok, result in parallel_bulk(es, load_data(wiki_data, "wiki_evidence"), chunk_size=500, request_timeout=60*3):
#     if ok is not True:
#             logging.error('Failed to import data')
#             logging.error(str(result))
#             errors_count += 1

#             if errors_count == errors_before_interrupt:
#                 logging.fatal('Too many import errors, exiting with error code')
#                 exit(1)

In [ ]:

# import spacy

# nlp = spacy.load("en_core_web_sm")

# def sentence_window(article, window=3, step=2): 
#     """ Generates a list of sentences of sliding size = window """
    
#     sents = list(nlp(article).sents)
    
#     if len(sents) == window:
#         yield sents

#     for i in range(0, len(sents)):
#         yield(sents[i:i + window])

# def load_data(docs, idx_):
#     """ Generates an evidence document to be inserted into ES Index """
#     for doc in docs:
#         idx, title, text = doc

#         for paragraph in sentence_window(text):
#             yield {
#                 "_index": idx_,
#                 "document": {
#                     "id": idx,
#                     "title": title,
#                     "text": paragraph
#                 }
#             }


In [ ]:
# import time
# from elasticsearch import helpers
# from elasticsearch.helpers import streaming_bulk
# from tqdm import tqdm
#
# "https://stackoverflow.com/questions/67522617/elasticsearch-bulk-insert-w-python-socket-timeout-error#:~:text=The%20connection%20to%20elasticsearch%20has,be%20handled%20as%20an%20error."
#
# def load_data(docs, idx_):
#     for doc in docs:
#         idx, title, text = doc
#         doc_ = {"id": idx, "title": title, "text": text}
#
#         yield {
#             "_index": idx_,
#             "document": doc
#         }
#
# # TODOs: Utils, Duration Function as Decorator
# es = wiki_ev
# #helpers.bulk(es, load_data(wiki_data, "wiki_evidence"), raise_on_error=False, chunk_size=500)
# errors_count = 0
# # TODOs: Increase Chunk Size, with extended Timeout + handelling
# for ok, result in streaming_bulk(es, load_data(wiki_data, "wiki_evidence"), chunk_size=500, request_timeout=60*3, yield_ok=yield_ok, refresh=refresh_index_after_insert):
#     if ok is not True:
#             logging.error('Failed to import data')
#             logging.error(str(result))
#             errors_count += 1
#
#             if errors_count == errors_before_interrupt:
#                 logging.fatal('Too many import errors, exiting with error code')
#                 exit(1)

In [ ]:
# for article in wiki_data:
#     _id, title, text = article
#     doc = {"id": _id, "title": title, "text": text}
#
#     wiki_ev.add_doc(doc)

In [ ]:
# https://github.com/elastic/elasticsearch-py/issues/297

In [ ]:
### DOCUMENT IMPORT: FULL-TEXTS ###
# from multiprocessing import Pool
# # from utils import get_contents
# import utils
# from tqdm import tqdm
# import json

# # TODOs: USE A GENERATOR OBJECT
# import spacy
# nlp = spacy.load("en_core_web_sm")
#
# p = Pool(8)
# files = [f for f in kw_sample]
#
# count = 0
# test = []
# with tqdm(total=len(files)) as pbar:
#     for documents in p.map(utils.get_contents_2, files):
#             for doc in documents:
#                 _id, title, text = doc
#
#             # count += 1
#             # doc_ = {"id": _id, "title": title, "text": text}
#
#             #wiki_ev.add_doc(doc)
#             test.append(doc_)
#     pbar.update()

In [ ]:
# from elasticsearch import Elasticsearch

# # INIT OBJECT

# # TODOs: Persist a Generator Object
# PORT = "http://localhost:9200"
# INDEX_NAME = "wiki_evidence"
# errors_before_interrupt = 5
# refresh_index_after_insert = False
# max_insert_retries = 3
# yield_ok = False

# wiki_ev = Elasticsearch(
#     PORT,
#     #http_auth=(es_api_user, es_api_password)
#     retry_on_timeout=True,  # should timeout trigger a retry on different node?
# )

# wiki_ev.elastic_index = INDEX_NAME

# wiki_ev

<Elasticsearch(['http://localhost:9200'])>

In [ ]:
# ### DOCUMENT IMPORT: SEGMENTED-TEXTS ###

# from multiprocessing import Pool
# # from utils import get_contents
# from tqdm import tqdm

# # p = Pool(8)
# # files = [f for f in kw_sample]
# #
# # count = 0
# # test = []
# # with tqdm(total=len(files)) as pbar:
# #     for documents in p.map(get_contents, files):
# #         for doc in documents:
# #             _id, title, text = doc
# #
# #             count += 1
# #             doc_ = {"id": _id, "title": title, "text": text}
# #
# #             #wiki_ev.add_doc(doc)
# #             test.append(doc)
# #     pbar.update()

In [ ]:
# ### QUERY DB ###
# import elastic_db
# # from elastic_db import ElasticDB
# #
# # # Params
# # PORT = "http://localhost:9200"
# # INDEX = "wiki_evidence"
# # DOC = "evidence"
# #
# # # Init Elasticsearch DB
# # wiki_ev_ = ElasticDB(elastic_port=PORT, elastic_index=INDEX, elastic_doc=DOC)
# #
# # results = wiki_ev_.search("exploitation a wider public debate indecency adult")
# # results

In [ ]:
### SQLITE LOAD ###
# from multiprocessing import Pool
# import utils
# from tqdm import tqdm
# import sqlite3

# import spacy
# import uuid

# nlp = spacy.load("en_core_web_sm")

# def paragraphs(document):
#     start = 0
#     document = nlp(document)
#     passages = []
#     for token in document:
#         if token.is_space and token.text.count("\n") > 1:
#             yield document[start:token.i]
#             start = token.i
#     yield document[start:]


# def get_contents(filename):
#     """Parse the contents of a file. Each line is a JSON encoded document."""
#     documents = []

#     with open(filename) as f:
#         for line in f:
#             doc = json.loads(line)

#             if doc["text"] == "": continue
#             if not doc: continue

#             passages = [str(i) for i in paragraphs(doc["text"])][0].split("\n")

#             for passage in passages:
#                 if len(passage) < 50:
#                     continue

#                 documents.append((str(uuid.uuid4()).replace('-',''), doc['id'], doc["title"], passage))

#     return documents

# save_path = "../data/wiki_evidence.db"
#
# p = Pool(8)
# files = [f for f in kw_sample]
#
# conn = sqlite3.connect(save_path)
# c = conn.cursor()
#
# documents = "documents"
# c.execute(f"CREATE TABLE documents (id PRIMARY KEY, id_, title, text);")
#
# count = 0
# step = 100
# batches = [files[i:i + step] for i in range(0, len(files), step)]
#
# for i, batch in enumerate(batches):
#     logger.info(f"[.... Batch #{i} .....]")
#     with tqdm(total=len(batch)) as pbar:
#         for document in tqdm(p.imap_unordered(get_contents, files)):
#             count += 1
#             for content in document:
#                 # _id, title, passage = content
#                 c.executemany("INSERT INTO documents VALUES (?,?,?,?)", (content,))
#
#         pbar.update()
#         logger.info(f"[Uploaded {count} documents]")
#
# conn.commit()
# conn.close()